In [11]:
import pandas as pd

In [12]:
data = {
    'image_id': ['img_001', 'img_002', 'img_003', 'img_004', 'img_005'],
    'pred': ['cat', 'dog', 'bird', 'cat', 'dog'],
    'votes_a': [10, 15, 7, 20, 5],
    'votes_b': [5, 10, 3, 8, 2],
    'votes_c': [2, 1, 0, 3, 1]
}

pd.DataFrame(data).set_index('image_id').to_csv("test_data.csv")

In [15]:
import subprocess

def add_a_vote(col, image_id):
    df = pd.read_csv("test_data.csv", index_col = "image_id")
    df.at[image_id,col] += 1
    df.to_csv("test_data.csv")
    subprocess.run(["git", "pull"])
    subprocess.run(['git', 'add', "test_data.csv"], check=True)
    subprocess.run(['git', 'commit', '-m', "update vote"], check=True)
    subprocess.run(["git","push"])
    
    

In [16]:
add_a_vote("votes_a","img_001")

hint: You have divergent branches and need to specify how to reconcile them.
hint: You can do so by running one of the following commands sometime before
hint: your next pull:
hint:
hint:   git config pull.rebase false  # merge
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint:
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.
fatal: Need to specify how to reconcile divergent branches.
fatal: pathspec 'labelling.ipynb.csv' did not match any files


CalledProcessError: Command '['git', 'add', 'labelling.ipynb.csv']' returned non-zero exit status 128.